# [Goolgule](https://www.goolgule.com/)

In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd 
import re

In [2]:
def isEnglish(s):
    """Takes a token and checks if the input token is english or other language
        Input (str): input token
        output (bool): true if english else false"""
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return bool(re.findall('[A-Za-z]', s))
    else:
        return True

In [33]:
url_list = list()
url_list.append({"category":"Editorials", "url":"https://www.goolgule.com/%e1%88%ad%e1%8b%95%e1%88%b0-%e1%8a%a0%e1%8a%95%e1%89%80%e1%8c%bd/page/","max_page":5})
url_list.append({"category":"News", "url":"https://www.goolgule.com/%e1%8b%9c%e1%8a%93/page/","max_page":79})
url_list.append({"category":"Social", "url":"https://www.goolgule.com/%e1%88%9b%e1%8a%85%e1%89%a0%e1%88%ab%e1%8b%8a/page/","max_page":18})
url_list.append({"category":"Religion", "url":"https://www.goolgule.com/%e1%88%bd%e1%8a%a8%e1%89%b3/page/","max_page":1})
url_list.append({"category":"Interviews", "url":"https://www.goolgule.com/%e1%8a%a5%e1%8a%93%e1%8b%8d%e1%8c%8b/page/","max_page":4})
url_list.append({"category":"Law", "url":"https://www.goolgule.com/%e1%8b%a8%e1%88%95%e1%8c%8d-%e1%8b%ab%e1%88%88%e1%88%85/page/","max_page":6})
url_list.append({"category":"Opinion", "url":"https://www.goolgule.com/%e1%8b%a8%e1%8a%94-%e1%88%83%e1%88%b3%e1%89%a5/page/","max_page":160})
url_list.append({"category":"Litrature", "url":"https://www.goolgule.com/%e1%8c%bd%e1%8d%88%e1%8a%aa%e1%8a%95/page/","max_page":27})


def spiderGoolgule(max_pages, page, cookies=None):
    data = list()
    for i in tqdm(range(2, max_pages)):
        url = page+str(i)+"/"
        if cookies != None:
            crawl_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, cookies=cookies)
        else:
            crawl_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        crawl_url.raise_for_status()
        soup = BeautifulSoup(crawl_url.text, 'lxml')
        body = soup.find("main", class_="content")         
        articles = soup.find_all("article", class_="post")        
        if articles == None:
            continue
        links = [x.header.a['href'] for x in articles]
        for link in links:
            if cookies != None:
                crawl_url = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'}, cookies=cookies)
            else:
                crawl_url = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            crawl_url.raise_for_status()
            soup = BeautifulSoup(crawl_url.text, 'lxml')
            body = soup.find("main", class_="content")
            
            if body == None:
                continue
            body = body.article
            header = body.find('header', class_="entry-header")
            if header != None:
                title = header.h1.text
                title = " ".join(title.split())
                meta_data = header.find("p", class_="entry-meta")
                if meta_data != None:
                    date = meta_data.find('time', class_="entry-time").text
            else:
                date, title = None, None
            content = " ".join(s for s in body.find("div", class_="entry-content").text.split() if not isEnglish(s))
            data.append([date, title, content])
    return data

In [ ]:
for site in url_list:
    data = spiderGoolgule(site['max_page'], site['url'])
    df = pd.DataFrame(data, columns =['date', 'title', 'content']) 
    df.to_csv("Goolgule_"+site['category']+".csv", encoding ='utf-16')

 36%|█████████████████████████████▊                                                    | 28/77 [10:25<18:40, 22.88s/it]